In [1]:
import threading
from time import sleep
import rclpy
from rclpy.node import Node
from visualization_msgs.msg import MarkerArray

# Initialize rclpy
rclpy.init()


In [2]:

# Create the node
node = rclpy.create_node("ipynb")

In [3]:
node.cur_semantic = None

In [4]:

# Define the callback
def semantic_cb(markers: MarkerArray):
    node.cur_semantic = markers
    print("Received semantic markers")
    # Destroy the subscription after receiving the first message
    node.destroy_subscription(node.subscription_semantic)

# Create the subscription
node.subscription_semantic = node.create_subscription(
    MarkerArray,
    "/object_markers",
    semantic_cb,
    5
)

In [ ]:
!ros2 topic echo "/object_markers"

In [6]:

# Define a function to spin in a separate thread
def spin_node():
    while rclpy.ok():
        rclpy.spin_once(node, timeout_sec=0.1)

# Start the spinning thread
spin_thread = threading.Thread(target=spin_node, daemon=True)
spin_thread.start()

# Main thread remains interactive
print("ROS 2 node is spinning in the background. You can interact with the notebook.")

# Example: Wait for the semantic data to be received
while node.cur_semantic is None:
    print("Waiting for semantic data...")
    sleep(0.5)

print("Semantic data received. Proceeding with other tasks.")


ROS 2 node is spinning in the background. You can interact with the notebook.Received semantic markers

Semantic data received. Proceeding with other tasks.


In [17]:
from visualization_msgs.msg import Marker

In [ ]:
mrkr = Marker()
mrkr.

In [37]:
node.cur_semantic.markers[0].pose.position

geometry_msgs.msg.Point(x=-1.3379569053649902, y=1.3440048694610596, z=0.6321040987968445)

In [39]:
markers = []
for marker in node.cur_semantic.markers:
    pose = {
        "x": marker.pose.position.y,
        "y": marker.pose.position.x,
        "z": marker.pose.position.z,
    }
    markers.append({
        "id": marker.id,
        "ns": marker.ns,
        "pose" : pose
    })
    # print(f"{marker.id} {marker.ns} {marker.color}")
markers

[{'id': 5,
  'ns': 'file',
  'pose': {'x': 1.3440048694610596,
   'y': -1.3379569053649902,
   'z': 0.6321040987968445}},
 {'id': 11,
  'ns': 'potted plant',
  'pose': {'x': 1.2849280834197998,
   'y': -0.40436631441116333,
   'z': 1.1766719818115234}},
 {'id': 14,
  'ns': 'box',
  'pose': {'x': 1.3699389696121216,
   'y': -0.4690001606941223,
   'z': 0.18483629822731018}},
 {'id': 17,
  'ns': 'paper box',
  'pose': {'x': 1.3560700416564941,
   'y': -0.4485590159893036,
   'z': 0.6978890299797058}},
 {'id': 25,
  'ns': 'paper box',
  'pose': {'x': 1.334167718887329,
   'y': -1.3583459854125977,
   'z': 0.22288921475410461}},
 {'id': 35,
  'ns': 'wall',
  'pose': {'x': 1.6510001420974731,
   'y': 0.9570003747940063,
   'z': 1.451615333557129}},
 {'id': 40,
  'ns': 'chair',
  'pose': {'x': -1.422675609588623,
   'y': -0.9733389616012573,
   'z': 0.5217414498329163}},
 {'id': 41,
  'ns': 'box',
  'pose': {'x': 1.2850040197372437,
   'y': -0.9090006351470947,
   'z': 0.21999992430210114}},

In [ ]:




class DummyVLM(Node):

    def __init__(self):
        super().__init__('ipynb')

        self.obj_mid_x = 3.3707286769194647
        self.obj_mid_y = -2.092013168894541
        self.obj_mid_z = 0.50101238489151
        self.obj_l = 2.8618994
        self.obj_w = 1.198852
        self.obj_h = 1.0225521
        self.object_heading = 0.001148942756478987

        self.waypoint_x = [-0.1767035871744156, 4.454692840576172]
        self.waypoint_y = [-1.3834829330444336, -4.007359027862549]
        self.waypoint_heading = [0.0, 0.0]
        self.waypoint_dis = 1.0

        self.vehicle_x = 0.0
        self.vehicle_y = 0.0
        self.question = ""

        self.traversible = None
        self.cur_semantic = None

        self.waypointID = 0

        self.subscription_pose = self.create_subscription(
            Odometry,
            "/state_estimation",
            self.pose_cb,
            5
        )
        self.subscription_question = self.create_subscription(
            String,
            "/challenge_question",
            self.question_cb,
            5
        )

        self.subscription_semantic = self.create_subscription(
            MarkerArray,
            "/object_markers",
            self.semantic_cb,
            5
        )

        self.subscription_traversible_area= self.create_subscription(
            PointCloud2,
            "/traversable_area",
            self.traversible_cb,
            5
        )

        self.waypoint_publisher = self.create_publisher(
            Pose2D,
            "/way_point_with_heading",
            5
        )
        self.object_marker_publisher = self.create_publisher(
            Marker,
            "/selected_object_marker",
            5
        )
        self.numerical_answer_publisher = self.create_publisher(
            Int32,
            "/numerical_response",
            5
        )

        self.timer = self.create_timer(
            0.01,
            self.timer_cb
        )
        self.waypoint_timer = None
        self.get_logger().info("Awaiting question...")

    def semantic_cb(self, markers: MarkerArray):
        self.cur_semantic = markers
    
    def traversible_cb(self, area: PointCloud2):
        self.traversible = area
        self.cleaned_traversible = point_cloud2.read_points_list(area)
        
